# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [28]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url,headers= headers)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img",  "input",]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""    
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [17]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [18]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [19]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [20]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [21]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [22]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [23]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/ByteDance/MegaTTS3',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/m

In [24]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [25]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [29]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse

In [48]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a brochure in Hindi about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [49]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [50]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\n1 day ago\n•\n51.1k\n•\n576\nQwen/Qwen2.5-Omni-7B\nUpdated\n8 days ago\n•\n97.7k\n•\n1.24k\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n12 days ago\n•\n161k\n•\n2.42k\nall-hands/openhands-lm-32b-v0.1\nUpdated\n4 days ago\n•\n4.16k\n•\n299\nopenfree/flux-chatgpt-ghibli-lora\nUpdated\n3 days ago\n•\n7.66k\n•\n300\nBrowse 1M+ models\nSpaces\nRunning\

In [51]:
def create_brochure_in_hindi(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [52]:
create_brochure_in_hindi("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


```markdown
# Hugging Face - ए.आई. समुदाय जो भविष्य का निर्माण कर रहा है

## हमारी कंपनी के बारे में
Hugging Face एक सहयोगी मंच है जहाँ मशीन लर्निंग समुदाय एकत्र होकर मॉडल, डेटासेट, और एप्लिकेशन पर काम करता है। हम ए.आई. एप्लिकेशन को बनाने, खोजने और साझा करने के लिए एक ऐसी जगह प्रदान करते हैं जहाँ आप 1 मिलियन से अधिक मॉडल ब्राउज़ कर सकते हैं।

## हमारी सेवाएँ
- **मॉडल्स**: उपयोग में आसानी के लिए 1 मिलियन से अधिक उच्च गुणवत्ता वाले ए.आई. मॉडल उपलब्ध।
- **डेटासेट्स**: विभिन्न मशीन लर्निंग कार्यों के लिए 250,000+ डेटासेट्स।
- **स्पेसेस**: आपके द्वारा बनाए गए या उपयोग किए जाने वाले एप्लिकेशनों के लिए एक इंटरैक्टिव मंच।

## क्यों चुनें Hugging Face?
- **ओपन-सोर्स प्लेटफार्म**: हम अपने समुदाय के साथ मिलकर मशीन लर्निंग टूलिंग का आधार बना रहे हैं।
- **एंटरप्राइज समाधान**: एंटरप्राइज-ग्रेड सुरक्षा और सहायता के साथ आपके टीम के लिए सबसे उन्नत प्लेटफार्म।
- **विभिन्न उद्योगों के उपयोगकर्ता**: हमारे व्यापक उपयोगकर्ताओं में Meta, Amazon, Google और Microsoft जैसी कंपनियाँ शामिल हैं, जो हमारी प्रौद्योगिकी का लाभ उठाते हैं।

## हमारी संस्कृति
Hugging Face में हम एक समुदाय का निर्माण कर रहे हैं जिसका लक्ष्य सबसे अच्छे ए.आई. उपकरणों का निर्माण करना है। हमारे यहाँ सहयोग, नवाचार और समावेश का आदान-प्रदान है। हम सभी व्यक्तियों को उनके विचारों और योगदान के लिए महत्व देते हैं।

## करियर और अवसर
Hugging Face में आपके लिए कई कैरियर के अवसर उपलब्ध हैं। हम ऐसे व्यक्तित्वों की तलाश कर रहे हैं जो मशीन लर्निंग, डेटा विज्ञान और ए.आई. विकास में रुचि रखते हैं। समूह में शामिल हो कर अपनी प्रतिभा से समुदाय को पुष्ट करें।

## संपर्क करें
अधिक जानने के लिए हमारी वेबसाइट पर जाएँ या सीधे हमसे संपर्क करें!

[हगिंग फेस की वेबसाइट](https://huggingface.co)

---

**Hugging Face - अपने विचारों को साझा करें, अपने भविष्य का निर्माण करें।**
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [53]:
def stream_brochure_in_hindi(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [54]:
stream_brochure_in_hindi("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}



# Hugging Face - भविष्य का निर्माण करने वाली AI समुदाय

## कंपनी के बारे में
Hugging Face एक सहयोगात्मक मंच है, जहाँ मशीन लर्निंग समुदाय मॉडल, डेटा सेट और एप्लिकेशन पर एक साथ काम करता है। हमारा उद्देश्य भविष्य में AI के विकास में योगदान देना है। 

### महत्वपूर्ण विशेषताएँ
- **मॉडल्स और डेटा सेट**: 1 मिलियन से अधिक मॉडल और 250,000 से अधिक डेटा सेट हमारे प्लेटफॉर्म पर उपलब्ध हैं।
- **हगिंगस्पेस**: सार्वजनिक और व्यावसायिक उपयोग के लिए असीमित मॉडल, डेटा सेट और एप्लिकेशन पर होस्टिंग और सहयोग।
- **एंटरप्राइज समाधान**: सुरक्षित और पेशेवर समर्थन के साथ उच्च-स्तरीय AI निर्माण के लिए एंटरप्राइज ग्रेड प्लेटफ़ॉर्म।

## कंपनी संस्कृति
Hugging Face एक खुली और समावेशी संस्कृति को बढ़ावा देता है। हम एक स्थाई, सहयोगी और नवाचार पर केंद्रित वातावरण बनाने में विश्वास करते हैं। हमारी टीम में 213 सदस्य हैं जो AI और मशीन लर्निंग में निष्पक्षता और उपलब्धता पर ध्यान केंद्रित करते हैं।

### ग्राहक और सहयोगी
हमारे प्लेटफ़ॉर्म का उपयोग 50,000 से अधिक संगठनों द्वारा किया जा रहा है, जिसमें प्रमुख कंपनियाँ जैसे कि:
- **Amazon**
- **Google**
- **Microsoft**
- **Meta**
  
यह कंपनियाँ हमारे समर्पित और सक्षम समुदाय से सहयोग की महत्ता को समझती हैं और हमारी सेवा का लाभ उठा रही हैं।

## करियर के अवसर
हमें प्रतिभाशाली व्यक्तियों की आवश्यकता है जो हमारे समुदाय में शामिल होना चाहते हैं। यदि आप मशीन लर्निंग, AI, या तकनीक में रुचि रखते हैं, तो Hugging Face आपके लिए एक आदर्श स्थान है।

### वर्तमान खुले पद
हम विभिन्न क्षेत्रों में रोजगार के अवसर प्रदान करते हैं। अपनी प्रतिभा को दिखाने के लिए हमारे साथ जुड़ें और एक नया अध्याय शुरू करें।

## हमसे संपर्क करें
अगर आप हमारे उत्पादों और सेवाओं के बारे में अधिक जानना चाहते हैं या फिर हमारे साथ जुड़ना चाहते हैं, तो हमारी वेबसाइट पर जाएं: [Hugging Face](https://huggingface.co)

**हमारा उद्देश्य** - AI समुदाय का विकास करना और भविष्य को बेहतर बनाना।



In [40]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face: A Brochure Full of AI and Dad Jokes!

Welcome to **Hugging Face**—the only place where AI developers hug it out! As a community at the forefront of machine learning, we’re building the future, one dataset and model at a time. Let’s dive into what makes us tick (and giggle)!

---

## About Us

At Hugging Face, we believe in collaboration, innovation, and a sprinkle of humor. Our mission? To make AI accessible and enjoyable for everyone!

**Why did the computer go to therapy?**  
*Because it had too many bytes!*

---

## Company Culture

We’re not your typical tech company! Picture a place where:

- **Teamwork** means building models together that can actually make a difference.
- **Creativity** flows as freely as coffee (and that's saying something).
- **Support** is more than just a help desk; it’s the warm embrace of a community.

**What do you call a programming language that’s confused?**  
*A ‘why’ language!*

---

## Our Customers

Over **50,000 organizations** trust us to power their ML needs, including giants like Google, Microsoft, and Amazon. It’s like the Avengers of AI—without the capes but with plenty of code!

**Why was the developer unhappy at their job?**  
*They wanted arrays!*

---

## Career Opportunities

Are you looking for a job that allows you to explore your creative side while working at the cutting edge of technology? Join us! We’re actively searching for innovative minds, passionate about AI.

**What do you call a music program that can’t find the beat?**  
*A flat-code generator!*

---

## Our Offerings

### 1. **Models**
- **1 Million+ Models** available! 
- Collaboration on models that you’d want to take home to meet your parents.

### 2. **Datasets**
- Access to over **250k datasets** to help you train your machine learning monsters!

### 3. **Spaces**
- Thousands of applications running freely in our supportive environment.

---

## Pricing

Starting at just **$0.60/hour for GPU**—which we promise isn’t a scam!  
It’s like getting a top-notch AI workout for the price of a coffee!

**Why don’t programmers like nature?**  
*It has too many bugs!*

---

## Join Us Today!

Sign up to be part of the Hugging Face community, where you can laugh, learn, and collaborate! Whether you’re a customer looking for killer models or a job seeker hoping to join our fun team, we can't wait to meet you!

---

### Contact Us

For inquiries, please visit our website: [Hugging Face](https://huggingface.co)  
Follow us on social media for the latest in AI (and dad jokes!).

---

**Why did the AI go broke?**  
*It lost its cache!*

Thank you for considering Hugging Face, where we’re not just building AI, we're building friendships... one pun at a time!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>